In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from ipywidgets import IntProgress
from IPython.display import display

In [5]:
from var_table import VarTable

In [6]:
from cost_calculator_factory import CostCalculatorFactory
from multibinary_converter import MultiBinaryConverter
from multidiscrete_converter import MultiDiscreteConverter
from pyomo_values_converter import PyomoValuesConverter
from fitness_calculator import FitnessCalculator

In [7]:
from genetic_multibinary_space_config import GeneticMultiBinarySpaceConfig
from genetic_multidiscrete_space_config import GeneticMultiDiscreteSpaceConfig
from genetic_config import GeneticConfig
from genetic_solver import GeneticSolver

In [8]:
# from stable_baselines3 import PPO
# from stable_baselines3 import A2C
# from stable_baselines3.common.env_checker import check_env

In [9]:
# from rl_multibinary_config import RlMultiBinaryConfig
# from rl_multidiscrete_config import RlMultiDiscreteConfig
# from rl_graphic import RlGraphic
# from rl_env import RlEnv
# from rl_solver import RlSolver

In [10]:
from model_builder import ModelBuilder
from solve_action import SolveAction

In [11]:
from random_solver import RandomSolver

In [12]:
from cost_calculator_factory import CostCalculatorFactory
from dependencies_calculator import DependenciesCalculator

In [13]:
M = 10 ** 10
dtype = np.int64
policy = 'MlpPolicy'

In [14]:
var_table = VarTable()

In [15]:
# algorithms = {
#     'A2C': A2C,
#     'PPO': PPO
# }

In [16]:
df = pd.read_csv('dataset.csv')

In [17]:
experiments_count = len(df)

In [18]:
class RandomSolveExperiment:

    def get_solver_name(self):
        return 'random'

    def make_experiment(self, input_data):
        k, T, sum_D, C, E = input_data
        cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)
        m = np.shape(T)[1]

        converter = MultiDiscreteConverter(m, k)
        fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
        random_solver = RandomSolver(m, k, fitness_calculator)
        solution, fitness, duration = random_solver.solve()

        return fitness, duration

In [19]:
class ConcreteSolveExperiment:
    
    def __init__(self, solver_name):
        self.solver_name = solver_name

    def get_solver_name(self):
        return self.solver_name

    def make_experiment(self, input_data):
        k, T, sum_D, C, E = input_data
        cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)

        converter = PyomoValuesConverter()
        fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
        model_builder = ModelBuilder(M, k, T, sum_D, C, E)
        model = model_builder.build()
        solver = SolveAction(model, self.solver_name)
        solve_result = solver.solve()
        instance = solve_result.instance
        duration = solve_result.duration
        A = instance.A
        fitness = fitness_calculator.calculate(A)

        return fitness, duration

In [20]:
class DiscreteGeneticExperiment:
    def __init__(self, solver_name, genetic_config):
        self.solver_name = solver_name
        self.genetic_config = genetic_config
    
    def get_solver_name(self):
        return self.solver_name

    def make_experiment(self, input_data):
        k, T, sum_D, C, E = input_data
        cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)

        m = np.shape(T)[1]
        converter = MultiDiscreteConverter(m, k)
        fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
        space_config = GeneticMultiDiscreteSpaceConfig(m, k)
        
        stub_progress = IntProgress(min=0, max=self.genetic_config.num_generations, value=0, description=str(self.genetic_config.num_generations), bar_style='success')
        # display(stub_progress)
        solver = GeneticSolver(self.genetic_config, space_config, fitness_calculator, stub_progress)
        solution, fitness, solution_idx, duration = solver.solve()
        fitness = float(fitness)

        return fitness, duration

In [21]:
class Experiments:
    def __init__(self, experiments_count):
        self.experiments_count = experiments_count

    def make(self, solve_experiment):
        solver_name = solve_experiment.get_solver_name()
        description = solver_name
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(175, self.experiments_count):
            input_data = self.read_input_data(experiment_number)
            fitness, duration = solve_experiment.make_experiment(input_data)
            config = {}
            result = self.solution_result_to_json(fitness, duration)
            self.write_config(experiment_number, solver_name, config, result)
            progress.value += 1
        
    def solution_result_to_json(self, fitness, duration):
        result = {
            'fitness': fitness,
            'duration': duration
        }
        return result
    
    def write_config(self, experiment_number, solver_name, config, result):
        serialized = {
            'config': config,
            'result': result
        }
        nested_directory = var_table.get_nested_directory(experiment_number)
        file_path = '{}/solutions.json'.format(nested_directory)
        file = open(file_path, 'r')
        content = file.read()
        json_content = json.loads(content)
        file.close()

        index_of_solver_name_in_json_array = self.find_index_of_solver_name_in_json_array(json_content, solver_name)
        if index_of_solver_name_in_json_array != -1:
            solver_solutions = json_content[index_of_solver_name_in_json_array]['solutions']
            index_of_solution_in_json_array = self.find_index_of_solution_in_json_array(solver_solutions, config)
            if index_of_solution_in_json_array != -1:
                existing_solution = solver_solutions[index_of_solution_in_json_array]
                existing_solution['result'] = result
            else:
                solver_solutions.append(serialized)
        else:
            solutions = [serialized]
            solver_solution = {
                'name': solver_name,
                'solutions': solutions
            }
            json_content.append(solver_solution)
        json_to_write = json.dumps(json_content, indent=2)
        file = open(file_path, 'w')
        file.write(json_to_write)
        file.close()

    def find_index_of_solver_name_in_json_array(self, json_content, solver_name):
        result = -1
        length = len(json_content)
        for i in range(length):
            json_object = json_content[i]
            name = json_object['name']
            if name == solver_name:
                result = i
        return result

    def find_index_of_solution_in_json_array(self, solutions, solution_config):
        result = -1
        length = len(solutions)
        for i in range(length):
            solution = solutions[i]
            config = solution['config']
            if config == solution_config:
                result = i
        return result
        
    def read_input_data(self, experiment_number):
        k = df['k'][experiment_number]
        nested_directory = var_table.get_nested_directory(experiment_number)
        T_file_path = '{}/T.csv'.format(nested_directory)
        D_file_path = '{}/D.csv'.format(nested_directory)
        sum_D_file_path = '{}/sum_D.csv'.format(nested_directory)
        C_file_path = '{}/C.csv'.format(nested_directory)
        E_file_path = '{}/E.csv'.format(nested_directory)
        T_df = pd.read_csv(T_file_path, index_col=False, header=None)
        D_df = pd.read_csv(D_file_path, index_col=False, header=None)
        sum_D_df = pd.read_csv(sum_D_file_path, index_col=False, header=None)
        C_df = pd.read_csv(C_file_path, index_col=False, header=None)
        E_df = pd.read_csv(E_file_path, index_col=False, header=None)
        T = T_df.to_numpy()
        D = D_df.to_numpy()
        sum_D = sum_D_df.to_numpy()
        C = C_df.to_numpy()
        E = E_df.to_numpy()
        return k, T, sum_D, C, E

In [22]:
def compute(experiments_count):
    experiments = Experiments(experiments_count)
    # random
    random_solve_experiment = RandomSolveExperiment()
    # experiments.make(random_solve_experiment)
    
    # solvers
    glpk_solve_experiment = ConcreteSolveExperiment('glpk')
    cbc_solve_experiment = ConcreteSolveExperiment('cbc')
    mpec_minlp_solve_eperiment = ConcreteSolveExperiment('mpec_minlp')
    ilogcp_solve_experiment = ConcreteSolveExperiment('ilogcp')

    # experiments.make(glpk_solve_experiment)
    experiments.make(cbc_solve_experiment)
    experiments.make(mpec_minlp_solve_eperiment)
    # experiments.make(ilogcp_solve_experiment)
    
    # genetic
    many_generations_genetic_config = GeneticConfig()
    two_points_genetic_config = GeneticConfig()
    large_population_genetic_config = GeneticConfig()

    many_generations_genetic_config.num_generations = 1000
    two_points_genetic_config.crossover_type = 'two_points'
    large_population_genetic_config.sol_per_pop = 400
    large_population_genetic_config.num_parents_mating = 200
    

    many_generations_genetic_experiment = DiscreteGeneticExperiment('many generations', many_generations_genetic_config)
    two_points_genetic_experiment = DiscreteGeneticExperiment('two points crossover', two_points_genetic_config)
    large_population_genetic_experiment = DiscreteGeneticExperiment('large population', large_population_genetic_config)

    # experiments.make(many_generations_genetic_experiment)
    # experiments.make(two_points_genetic_experiment)
    # experiments.make(large_population_genetic_experiment)

In [23]:
compute(experiments_count)

IntProgress(value=0, bar_style='success', description='cbc', max=194)

IntProgress(value=0, bar_style='success', description='mpec_minlp', max=194)

the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
the transformation: None
